In [1]:
import os
import pandas as pd
import numpy as np
from nilearn.connectome import ConnectivityMeasure
import matplotlib.pyplot as plt

In [2]:
def extract_data():
    data = []
    for f in os.listdir("PCA_results/"):
        if not f.startswith(".") and f.startswith("sub"):
            data.append(pd.read_csv(os.path.join("PCA_results", f)).values)
    return np.stack(data)

In [3]:
data = extract_data()

## GMM Try

In [4]:
from sklearn.mixture import GaussianMixture

In [174]:
def GMM_fit():
    result = {}
    for d in range(6):
        visual_data = data[:, :, d]
        likelihood = np.zeros(len(data) * 150).reshape(len(data), 150)
        for i in range(len(data)):
            previous = 0
            for j in range(1, 151):
                gmm = GaussianMixture(n_components=j, covariance_type='spherical')
                gmm.fit(visual_data[i].reshape(-1, 1))
                new = gmm.score(visual_data[i].reshape(-1, 1))
                likelihood[i, j - 1] = new - previous
                previous = new
            print(".", end='')
        result[str(d)] = np.argmax(likelihood, axis=1)
        print("one finished")
    df = pd.DataFrame(result)
    df.to_csv("GMM_fit.csv")
    return df

In [175]:
result_df = GMM_fit()

............................one finished
............................one finished
............................one finished
............................one finished
............................one finished
............................one finished


In [7]:
from scipy.stats import ttest_1samp

In [326]:
ttest_1samp(result_df['3'], 120)

Ttest_1sampResult(statistic=-2.0196643479643, pvalue=0.05344483488971911)

In [6]:
manual = data[:, :, 0]
visual = data[:, :, 3]

In [22]:
gmm = GaussianMixture(n_components=120, covariance_type='spherical')
gmm.fit(np.array(list(zip(range(len(visual[10])), visual[10]))))

GaussianMixture(covariance_type='spherical', init_params='kmeans',
        max_iter=100, means_init=None, n_components=120, n_init=1,
        precisions_init=None, random_state=None, reg_covar=1e-06,
        tol=0.001, verbose=0, verbose_interval=10, warm_start=False,
        weights_init=None)

In [23]:
visual_e = gmm.means_[:, 0]

In [24]:
gmm.covariances_.mean()

5.819754429143024

In [210]:
gmm = GaussianMixture(n_components=120, covariance_type='spherical')
gmm.fit(np.array(list(zip(range(len(manual[10])), manual[10]))))

GaussianMixture(covariance_type='spherical', init_params='kmeans',
        max_iter=100, means_init=None, n_components=120, n_init=1,
        precisions_init=None, random_state=None, reg_covar=1e-06,
        tol=0.001, verbose=0, verbose_interval=10, warm_start=False,
        weights_init=None)

In [211]:
manual_e = gmm.means_[:, 0]

In [212]:
manual_e.sort()